<a href="https://colab.research.google.com/github/rudysemola/IFTS-MELA-ml-lab/blob/main/DeepRL_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep RL usando Stable Baselines e OpenAI Gym 

In questo notebook imparerai le basi pratiche sul deep RL usando la libreria stable baselines (RL) e OpenAI Gym*. 
In particolare, scelto l'ambiente su cui opera l'agente e la strategia deep RL vedremo come: 
- Crearlo con Stable Baselines
- Addestrarlo 
- Valutarlo

*Open AI gym for RL environments and tasks

## Installazione delle dipendenze e Stable Baseline
```
pip install stable-baselines3[extra]
```

Vediamolo in pratica

In [ ]:
!apt-get install ffmpeg freeglut3-dev xvfb  # For visualization
!pip install stable-baselines3[extra]
!pip install pyglet==1.4

In [ ]:
import stable_baselines3
stable_baselines3.__version__

'1.6.2'

## Importiamo!

Stable-Baselines funziona su ambienti che seguono l'interfaccia OpenAI [gym](https://www.gymlibrary.dev/). 

per questo motivo 

In [ ]:
import gym
import numpy as np

Adesso si importa il modello deep RL.
Scegliamo un modello model-free: Proximal Policy Optimization ([PPO](https://openai.com/blog/openai-baselines-ppo/)).


In [ ]:
from stable_baselines3 import PPO

La prossima cosa da importare è la classe di policy che verrà usata per creare le reti (per le funzione policy e funzione value). 

Nota che tali funzioni qui vengono approssimate attraverso l'uso delle reti neurali (MLP=Multi-Layer-Perceptron)

In [ ]:
from stable_baselines3.ppo.policies import MlpPolicy

**NOTE**

[Proximal Policy Optimization](https://stable-baselines.readthedocs.io/en/master/modules/ppo2.html) è un metodo Actor-Critic: utilizza una value function per migliorare la discesa del gradiente della policy.

Combina l'idea di [A2C](https://stable-baselines.readthedocs.io/en/master/modules/a2c.html) e [TRPO](https://stable-baselines.readthedocs.io/en/master/modules/trpo.html)

PPO è un algoritmo on-policy, le traiettorie utilizzate per aggiornare le reti devono essere raccolte utilizzando la politica più recente.
Di solito è meno sample efficient rispetto alle strategie off-policy come [DQN](https://stable-baselines.readthedocs.io/en/master/modules/dqn.html), [SAC](https://stable-baselines.readthedocs.io/en/master/modules/sac.html) o [TD3](https://stable-baselines.readthedocs.io/en/master/modules/td3.html).
Tuttavia è molto più veloce nell'addestramento.

PPO è diventato l'algoritmo di apprendimento per rinforzo predefinito in OpenAI e Stable Baseline grazie alla sua facilità d'uso e alle buone prestazioni.


## Creazione del Gym env e istanziare agente

Per questo esempio utilizzeremo l'ambiente *CartPole*, un classico problema di controllo.

Molto spesso i problemi di RL partono da una definizione narrativa e meno formale di quella vista a lezione. 
Ecco un esempio per CartPole a seguire

"Un palo è attaccato da un giunto non azionato a un carrello, che si muove lungo un binario senza attrito. Il sistema è controllato applicando una forza di +1 o -1 al carrello. Il pendolo inizia in posizione verticale e l'obiettivo è impedirne la caduta. Viene fornita una ricompensa di +1 per ogni passo in cui il palo rimane in piedi."

Cartpole environment: [https://gym.openai.com/envs/CartPole-v1/](https://gym.openai.com/envs/CartPole-v1/)

![Cartpole](https://cdn-images-1.medium.com/max/1143/1*h4WTQNVIsvMXJTCpXm_TAw.gif)


**Scelte di design**

Abbiamo scelto *MlpPolicy* perché l'input di CartPole è un vettore di feature, qui non abbiamo immagini o serie temporali!

Il tipo di azione da utilizzare (discreta/continua) verrà automaticamente dedotto dallo spazio delle azioni dell'ambientale (environment action space).


In [ ]:
env = gym.make('CartPole-v1')
model = PPO(MlpPolicy, env, verbose=1)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


(PLUS) come funziona una funzione di supporto per valutare l'agente?

Ecco esempio:

In [ ]:
def evaluate(model, num_episodes=100):
    """
    Evaluate a RL agent
    :param model: (BaseRLModel object) the RL Agent
    :param num_episodes: (int) number of episodes to evaluate it
    :return: (float) Mean reward for the last num_episodes
    """
    # This function will only work for a single Environment
    env = model.get_env()
    all_episode_rewards = []
    for i in range(num_episodes):
        episode_rewards = []
        done = False
        obs = env.reset()
        while not done:
            # _states are only useful when using LSTM policies
            action, _states = model.predict(obs)
            # here, action, rewards and dones are arrays
            # because we are using vectorized env
            obs, reward, done, info = env.step(action)
            episode_rewards.append(reward)

        all_episode_rewards.append(sum(episode_rewards))

    mean_episode_reward = np.mean(all_episode_rewards)
    print("Mean reward:", mean_episode_reward, "Num episodes:", num_episodes)

    return mean_episode_reward

Valutiamo l'agente non addestrato (un agente random).

In [ ]:
from stable_baselines3.common.evaluation import evaluate_policy

In [ ]:
# Use a separate environement for evaluation
eval_env = gym.make('CartPole-v1')

# Random Agent, before training
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=100)

print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

/usr/local/lib/python3.7/dist-packages/stable_baselines3/common/evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


mean_reward:19.13 +/- 8.26


Una volta addestrato, riusciremo a fare di meglio?

## Addestriamo l'agente e valutiamolo

In [ ]:
# Train the agent for 10000 steps
model.learn(total_timesteps=10000)

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 21.2     |
|    ep_rew_mean     | 21.2     |
| time/              |          |
|    fps             | 835      |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 26.4        |
|    ep_rew_mean          | 26.4        |
| time/                   |             |
|    fps                  | 630         |
|    iterations           | 2           |
|    time_elapsed         | 6           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.009202665 |
|    clip_fraction        | 0.0957      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | -0.00153    |
|    learning_rate        | 0.

In [ ]:
# Evaluate the trained agent
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=100)

print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

mean_reward:415.41 +/- 97.07


A quanto pare l'allenamento è andato bene, la ricompensa media è aumentata molto!


### Fun Moment: video delle prestazioni dell'agente :)

Adesso proviamo a comprendere quello che riesce a fare il nostro modello preparando una registrazione!



In [ ]:
# Set up fake display; otherwise rendering will fail
import os
os.system("Xvfb :1 -screen 0 1024x768x24 &")
os.environ['DISPLAY'] = ':1'

In [ ]:
import base64
from pathlib import Path

from IPython import display as ipythondisplay

def show_videos(video_path='', prefix=''):
  """
  Taken from https://github.com/eleurent/highway-env

  :param video_path: (str) Path to the folder containing videos
  :param prefix: (str) Filter the video, showing only the only starting with this prefix
  """
  html = []
  for mp4 in Path(video_path).glob("{}*.mp4".format(prefix)):
      video_b64 = base64.b64encode(mp4.read_bytes())
      html.append('''<video alt="{}" autoplay 
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{}" type="video/mp4" />
                </video>'''.format(mp4, video_b64.decode('ascii')))
  ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))

Registreremo un video usando  [VecVideoRecorder](https://stable-baselines.readthedocs.io/en/master/guide/vec_envs.html#vecvideorecorder).

In [ ]:
from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv

def record_video(env_id, model, video_length=500, prefix='', video_folder='videos/'):
  """
  :param env_id: (str)
  :param model: (RL model)
  :param video_length: (int)
  :param prefix: (str)
  :param video_folder: (str)
  """
  eval_env = DummyVecEnv([lambda: gym.make('CartPole-v1')])
  # Start the video at step=0 and record 500 steps
  eval_env = VecVideoRecorder(eval_env, video_folder=video_folder,
                              record_video_trigger=lambda step: step == 0, video_length=video_length,
                              name_prefix=prefix)

  obs = eval_env.reset()
  for _ in range(video_length):
    action, _ = model.predict(obs)
    obs, _, _, _ = eval_env.step(action)

  # Close the video recorder
  eval_env.close()

In [ ]:
record_video('CartPole-v1', model, video_length=500, prefix='ppo2-cartpole')

Saving video to /content/videos/ppo2-cartpole-step-0-to-step-500.mp4


Vediamolo!

In [ ]:
show_videos('videos', prefix='ppo2')

In questo modo abbiamo visto in azione il nostro agente su un ambiente simulato che risolve un problema di controllo 'CartPole'. 

### Bonus: Addestra un modello RL in una riga

Ora che abbiamo capito come si opera per un generico problema di RL usando strategie di Deep RL siamo pronti a scoprire che...

Stable Baseline ci consente di ridurre il codice da creare per eseguire questi passaggi in una riga!

In [ ]:
model = PPO('MlpPolicy', "CartPole-v1", verbose=1).learn(1000)

## Addestriamo un altro agente: DQN

Nell'esempio precedente, abbiamo utilizzato PPO, uno dei tanti algoritmi forniti da stable-baselines.

Adesso operiamo con un altro algoritmo [Deep Q-Network agent (DQN)](https://stable-baselines.readthedocs.io/en/master/modules/dqn.html), e proviamo a vedere i possibili miglioramenti forniti dalle sue estensioni (Double-DQN, Dueling-DQN, Prioritized Experience Replay).

Il punto essenziale di questa sezione è mostrare quanto sia semplice modificare gli iperparametri delle strategie RL.

Il vantaggio principale di stable-baselines è che fornisce un'interfaccia comune per utilizzare gli algoritmi RL, quindi il codice sarà abbastanza simile.

In [ ]:
# Same as before we instantiate the agent along with the environment
from stable_baselines3 import DQN

# Note that the MlpPolicy of DQN is different from the one of PPO
# but stable-baselines handles that automatically if you pass a string
dqn_model = DQN('MlpPolicy', 'CartPole-v1', verbose=1)

Using cuda device
Creating environment from the given name 'CartPole-v1'
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [ ]:
# Random Agent, before training
mean_reward_before_train = evaluate(dqn_model, num_episodes=100)

Mean reward: 9.26 Num episodes: 100


In [ ]:
# Train the agent for 10000 steps
dqn_model.learn(total_timesteps=10000, log_interval=10)

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 22.4     |
|    ep_rew_mean      | 22.4     |
|    exploration_rate | 0.787    |
| time/               |          |
|    episodes         | 10       |
|    fps              | 7438     |
|    time_elapsed     | 0        |
|    total_timesteps  | 224      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 22.2     |
|    ep_rew_mean      | 22.2     |
|    exploration_rate | 0.578    |
| time/               |          |
|    episodes         | 20       |
|    fps              | 7618     |
|    time_elapsed     | 0        |
|    total_timesteps  | 444      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 24       |
|    ep_rew_mean      | 24       |
|    exploration_rate | 0.317    |
| time/               |          |
|    episodes       

In [ ]:
# Evaluate the trained agent
mean_reward = evaluate(dqn_model, num_episodes=100)

Mean reward: 9.73 Num episodes: 100


## Esercizi

Sperimentare sullo stesso task ed ambiente altre strategie di Deep RL usando lo Zoo:  https://stable-baselines3.readthedocs.io/en/master/guide/algos.html

In [ ]:
#TODO

## Your Turn!

Sperimentare 
- Usando i diversi ambienti forniti su OpenAI Gym
- Utilizzare diversi agenti Deep RL per diversi problemi 

**NOTA**: non tutti possono essere applicati!

In [ ]:
#TODO

# Materiale Aggiuntivo

### Stable Baseline
Slides: https://araffin.github.io/slides/rl-tuto-jnrr19/#/ 

Github repo: https://github.com/araffin/rl-tutorial-jnrr19

Stable-Baselines: https://github.com/hill-a/stable-baselines

Documentation: https://stable-baselines.readthedocs.io/en/master/

Tutorials: https://stable-baselines.readthedocs.io/en/master/guide/examples.html

RL Baselines zoo: https://github.com/araffin/rl-baselines-zoo (raccolta di agenti di apprendimento per rinforzo pre-addestrati che utilizzano Stable-Baselines)

Medium article: [https://medium.com/@araffin/stable-baselines-a-fork-of-openai-baselines-df87c4b2fc82](https://medium.com/@araffin/stable-baselines-a-fork-of-openai-baselines-df87c4b2fc82)

### DQN & Famiy
DQN paper: https://arxiv.org/abs/1312.5602

Dueling DQN: https://arxiv.org/abs/1511.06581

Double-Q Learning: https://arxiv.org/abs/1509.06461

Prioritized Experience Replay: https://arxiv.org/abs/1511.05952
